In [4]:
import numpy as np
import pandas as pd
import os

In [3]:
savee = 'ALL'

In [5]:
savee = os.listdir(savee)

In [9]:
print(savee)

['JK_sa01.wav', 'JK_sa15.wav', 'DC_n13.wav', 'DC_su09.wav', 'DC_n07.wav', 'JK_n20.wav', 'JK_n08.wav', 'JE_sa08.wav', 'JK_f15.wav', 'JK_f01.wav', 'KL_sa13.wav', 'KL_sa07.wav', 'JK_d03.wav', 'DC_h01.wav', 'DC_h15.wav', 'DC_h14.wav', 'JK_d02.wav', 'KL_sa06.wav', 'KL_sa12.wav', 'JK_f14.wav', 'JE_sa09.wav', 'JK_n09.wav', 'JK_n21.wav', 'DC_n06.wav', 'DC_n12.wav', 'DC_su08.wav', 'JK_sa14.wav', 'JK_sa02.wav', 'DC_n04.wav', 'DC_n10.wav', 'JK_n23.wav', 'JK_f02.wav', 'KL_sa04.wav', 'KL_sa10.wav', 'JK_d14.wav', 'DC_h02.wav', 'DC_h03.wav', 'JK_d15.wav', 'KL_sa11.wav', 'JK_d01.wav', 'KL_sa05.wav', 'JK_f03.wav', 'JK_n22.wav', 'DC_n11.wav', 'DC_n05.wav', 'JK_sa03.wav', 'JK_sa13.wav', 'JK_sa07.wav', 'DC_n29.wav', 'DC_n01.wav', 'DC_n15.wav', 'JK_n26.wav', 'JK_f07.wav', 'JK_f13.wav', 'JK_h08.wav', 'KL_sa01.wav', 'JK_d05.wav', 'KL_sa15.wav', 'JK_d11.wav', 'DC_f08.wav', 'DC_h13.wav', 'KL_a09.wav', 'DC_h07.wav', 'DC_h06.wav', 'DC_h12.wav', 'KL_a08.wav', 'DC_f09.wav', 'JK_d10.wav', 'KL_sa14.wav', 'JK_d04.wav

In [11]:
emotion = []
for wav in savee:
    w = wav.split('_')[1]
    if w[0:2] == 'sa':
        emotion.append('sad')
    elif w[0] == 'n':
        emotion.append('neutral')
    elif w[0:2] == 'su':
        emotion.append('surprised')
    elif w[0] == 'f':
        emotion.append('fear')
    elif w[0] == 'd':
        emotion.append('disgust')
    elif w[0] == 'h':
        emotion.append('happy')
    elif w[0] =='a':
        emotion.append('anger')
print(len(emotion))

480


In [ ]:
emotion_df = pd.DataFrame(emotion, columns=['Emotions']